# Notebook 1 · Classic RAG Baseline

This notebook re-creates the foundational retrieval-augmented generation (RAG) setup that powers the `example_app/` CLI. You will:

1. Load the FAISS vector store that indexes the NimbusWorkspaces help center.
2. Compose a `RetrievalQA` chain that fetches documents and synthesises an answer with `gpt-4.1-mini`.
3. Inspect latency and retrieved sources to build intuition for later agentic extensions.

> ℹ️ **Pre-requisite** — run `python example_app/ingest.py` before executing the cells so the FAISS store is available.

In [ ]:
from pprint import pprint

from shared import (
    DEFAULT_MODEL,
    RetrievalContext,
    build_baseline_chain,
    build_retrieval_context,
    pretty_print_json,
    time_execution,
)


In [ ]:
context: RetrievalContext = build_retrieval_context(top_k=4)
chain = build_baseline_chain(context.retriever)
print(f"Using chat model: {DEFAULT_MODEL}")


In [ ]:
question = "How can I reset the admin password for my workspace?"

result_payload = time_execution(chain.invoke, {"query": question})
print(f"Elapsed: {result_payload['elapsed_seconds']:.2f}s")
answer = result_payload["result"]["result"]
print(answer)


In [ ]:
retrieved_docs = context.retriever.get_relevant_documents(question)
print(f"Top {len(retrieved_docs)} supporting documents:")
for idx, doc in enumerate(retrieved_docs, start=1):
    title = doc.metadata.get("title") or doc.metadata.get("source", "doc")
    snippet = doc.page_content[:400].replace('\n', ' ')
    print(f"\n[{idx}] {title}
{snippet}...")


## Reflection

* Does the retrieved evidence fully answer the question?
* What happens if you increase `top_k` or change the question phrasing?
* Try swapping the model to `gpt-4.1-mini` or `gpt-4.1` for a quality comparison.